This notebook demonstrates the WorldPeatland site-level peat health indicators.

See `site-indicators.md` for more information about the data that drives this visualization.

```bash
$ tree .
.
├── info.json
├── peat_extent.geojson
├── time_series.h5
└── variable_loading
    ├── expert.json
    ├── svd.json
    └── ...
```

In [ ]:
import os
from typing import Annotated

import holoviews as hv
import pandas as pd
import panel as pn
import param
import pydantic
import shapely

import utils.climatology

hv.extension("bokeh")
pn.extension()

In [ ]:
DIRECTORY = "synthetic-indicators/degero/extent-1"

load data

In [ ]:
assert os.path.isdir(DIRECTORY)

In [ ]:
info_file = os.path.join(DIRECTORY, "info.json")
extent_file = os.path.join(DIRECTORY, "peat_extent.geojson")
timeseries_file = os.path.join(DIRECTORY, "time_series.h5")
variable_loading_dir = os.path.join(DIRECTORY, "variable_loading")

In [ ]:
assert os.path.isdir(variable_loading_dir)

In [ ]:
class InfoModel(pydantic.BaseModel):
    name: str
    description: str
    default_variable_loading: str

In [ ]:
with open(info_file, "r") as f:
    info = InfoModel.model_validate_json(f.read())

In [ ]:
info

In [ ]:
with open(extent_file, "r") as f:
    peat_extent = shapely.from_geojson(f.read())

In [ ]:
data = pd.read_hdf(timeseries_file, key="data")
variance = pd.read_hdf(timeseries_file, key="variance")

In [ ]:
data.head()

In [ ]:
# constrain variable loading l_v to [-1, 1]
type loading = Annotated[float, pydantic.Field(ge=-1.0, le=1.0)]


class VariableLoadingModel(pydantic.BaseModel):
    name: str
    description: str
    optimal_values: dict[str, float]
    variable_loadings: dict[str, loading]

In [ ]:
# mapping from file name to instance of VariableLoadingModel
variable_loadings: dict[str, VariableLoadingModel] = {}
for file_name in os.listdir(variable_loading_dir):
    if not file_name.endswith(".json"):
        continue
    with open(os.path.join(variable_loading_dir, file_name), "r") as f:
        variable_loadings[file_name] = VariableLoadingModel.model_validate_json(f.read())

In [ ]:
variable_loadings

In [ ]:
# pick the default variable loading
assert info.default_variable_loading in variable_loadings
current_variable_loading = variable_loadings[info.default_variable_loading]

Container per variable

In [ ]:
class Variable(pn.viewable.Viewer):
    """
    Container for a time series variable.

    All pandas Series and DataFrames should share a common DatetimeIndex.

    Transforms the time series to abs(data - optimal_value) if `transform` is True.
    This is useful if the variable is not monotonically correlated with peat health.
    """

    name: str = param.String(allow_None=False)  # type: ignore

    data: pd.Series = param.ClassSelector(class_=pd.Series, allow_None=False, constant=True, doc="Time series")  # type: ignore
    variance: pd.Series = param.ClassSelector(
        class_=pd.Series, allow_None=False, constant=True, doc="Variance of the time series"
    )  # type: ignore

    transform: bool = param.Boolean(label="Apply transformation relative to optimal value")  # type: ignore
    optimal_value: float = param.Number()  # type: ignore

    time_series: pd.Series = param.ClassSelector(
        class_=pd.Series, allow_None=False, constant=True, doc="Time series, transformed if requested"
    )  # type: ignore
    climatology_bounds: pd.DataFrame = param.ClassSelector(class_=pd.DataFrame, allow_None=False, constant=True)  # type: ignore
    z_score: pd.Series = param.ClassSelector(
        class_=pd.Series, allow_None=False, constant=True, doc="Standard anomaly of the time series"
    )  # type: ignore

    @param.depends("transform", "optimal_value", watch=True, on_init=True)
    def transform_time_series(self):
        """
        If `transform` is True, calculate the absolute difference from `optimal_value`.
        """
        if self.transform:
            time_series = (self.data - self.optimal_value).abs()
        else:
            time_series = self.data

        # 365-day climatology
        climatology = utils.climatology.daily_climatology(time_series, self.variance)
        climatology_bounds = utils.climatology.get_climatology_bounds(time_series.index, climatology)  # type: ignore
        z_score = utils.climatology.standard_anomaly(time_series, climatology)

        with param.edit_constant(self):
            # single transaction update
            self.param.update(
                time_series=time_series,
                climatology_bounds=climatology_bounds,
                z_score=z_score,
            )

    def widgets(self) -> pn.Param:
        # only show optimal_value if transform is True
        return pn.Param(
            self,
            parameters=["transform", "optimal_value"],
            show_name=False,
            widgets={
                "optimal_value": {"visible": self.param.transform.rx()}
            }
        )

    def _fix_index_names(self):
        """
        Ensure that all pandas objects have index name "time".
        This is necessary for HoloViews to correctly map the index to a key dimension.
        """
        for pandas_obj in [self.data, self.variance, self.time_series, self.climatology_bounds, self.z_score]:
            if not isinstance(pandas_obj.index, pd.DatetimeIndex):
                raise ValueError("Index must be a DatetimeIndex")
            if pandas_obj.index.name != "time":
                pandas_obj.index.name = "time"

    def original_data_view(self):
        """
        HoloViews plot of the original time series (not transformed relative to the optimal value).
        """
        self._fix_index_names()
        
        curve = hv.Curve(
            self.data,
            kdims=["time"],
            vdims=[self.name],
        )

        scatter = hv.Scatter(
            self.data, 
            kdims=["time"],
            vdims=[self.name],
        )
        scatter.opts(size=4)

        std = self.variance**0.5

        error_bars = hv.ErrorBars(
            (self.data.index, self.data, std),
            kdims=["time"],
            vdims=[self.name, f"{self.name}_std"],
        )

        overlay = error_bars * curve * scatter
        overlay.opts(
            title=f"{self.name} with error bars at 1 standard deviation",
            xlabel="date",
            ylabel=self.name,
        )

        return overlay

    @param.depends("transform", "time_series", "climatology_bounds", watch=False)
    def time_series_view(self):
        """
        Holoviews plot of the time series,
        potentially transformed relative to the optimal value (depending on `transform`).

        Shows the climatology envelope.
        """
        self._fix_index_names()

        if self.transform:
            title = f"{self.name} (relative to optimal) with climatology at 1 standard deviation"
        else:
            title = f"{self.name} with climatology at 1 standard deviation"

        curve = hv.Curve(
            self.time_series,
            kdims=["time"],
        )
        curve.opts(framewise=True)  # allow ylims to update

        scatter = hv.Scatter(
            self.time_series,
            kdims=["time"],
        )
        scatter.opts(size=4)

        area = hv.Area(
            # fill between (X, Y1, Y2)
            (
                self.climatology_bounds.index,
                self.climatology_bounds["lower bound"],
                self.climatology_bounds["upper bound"],
            ),
            kdims=["time"],
            vdims=["lower bound", "upper bound"],
        )
        area.opts(alpha=0.4)

        overlay = area * curve * scatter
        overlay.opts(
            title=title,
            xlabel="date",
            ylabel=self.name,
        )

        return overlay

    @param.depends("z_score", watch=False)
    def z_score_view(self):
        """
        Holoviews plot of the standard anomaly (z-score) of the time series.
        """
        self._fix_index_names()

        curve = hv.Curve(
            self.z_score,
            kdims=["time"],
        )

        scatter = hv.Scatter(
            self.z_score,
            kdims=["time"],
        )
        scatter.opts(
            size=4,
        )

        overlay = curve * scatter
        overlay.opts(
            title=f"{self.name} z-score",
            xlabel="date",
            ylabel=f"{self.name}",
        )

        return overlay

    def __panel__(self) -> pn.Column:
        """
        Panel layout for the Variable.
        """
        # IMPORTANT: plot size and layout must be set consistently twice!
        # 1. on the HoloViews object .opts(responsive=True)
        # 2. on the pn.pane.HoloViews(sizing_mode=...)

        time_series_view = hv.DynamicMap(self.time_series_view)
        time_series_view.opts(
            height=300,
            min_width=600,
            responsive=True,
        )

        z_score_view = hv.DynamicMap(self.z_score_view)
        z_score_view.opts(
            height=300,
            min_width=600,
            responsive=True,
        )

        return pn.Column(
            pn.Row(
                pn.pane.HoloViews(
                    self.original_data_view(),
                    height=300,
                    min_width=600,
                    sizing_mode="stretch_width",
                )
            ),
            pn.Row(self.widgets()),
            pn.Row(
                pn.pane.HoloViews(
                    time_series_view,
                    height=300,
                    min_width=600,
                    sizing_mode="stretch_width",
                )
            ),
            pn.Row(
                pn.pane.HoloViews(
                    z_score_view,
                    height=300,
                    min_width=600,
                    sizing_mode="stretch_width",
                ),
            ),
            width_policy="max",
        )

In [ ]:
var_name = "variable_1"

var = Variable(
    name=var_name,
    data=data[var_name],
    variance=variance[var_name],
    optimal_value=current_variable_loading.optimal_values.get(var_name, 0.0),
    transform=var_name in current_variable_loading.optimal_values,
)

In [ ]:
var